In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import urllib.parse

# — CONFIG —
TEXTS_CSV     = "post_match_texts.csv"
RESPONSES_CSV = "app_responses.csv"
MAX_RATINGS   = 20

# — LOAD DATA —
texts_df = pd.read_csv(TEXTS_CSV, encoding='cp1252', engine='python')
try:
    resp_df = pd.read_csv(RESPONSES_CSV)
except FileNotFoundError:
    resp_df = pd.DataFrame(columns=['text_id','sentiment'])

def get_unseen():
    seen = set(resp_df['text_id'])
    return texts_df[~texts_df['text_id'].isin(seen)]

# — TASK DESCRIPTION WIDGET —
task = widgets.HTML(
    value="""
    <div style="margin-bottom:1em; padding:10px; border:1px solid #ccc; border-radius:4px;">
      <strong>Task:</strong><br>
      Please read each text carefully and rate the overall sentiment of the manager’s statement as positive or negative.
      Your rating should reflect the manager’s expressed tone, <em>not</em> your judgment of the match.
    </div>
    """
)

# — MAIN WIDGETS —
out            = widgets.HTML()
sentiment      = widgets.RadioButtons(
    options=[
        'Strongly positive sentiment (clear optimism, satisfaction, praise)',
        'Mildly positive sentiment (generally positive, slight reservations)',
        'Neutral or unclear sentiment',
        'Mildly negative sentiment (general disappointment, frustration)',
        'Strongly negative sentiment (clear criticism, significant disappointment)'
    ],
    description='Your rating:',
    layout=widgets.Layout(width='80%')
)
btn_submit     = widgets.Button(description='Submit & Next')
download_link  = widgets.HTML()

# — DATA-URI DOWNLOAD HELPERS (unchanged) —
def make_csv_data_uri(df, filename="app_responses.csv"):
    csv_text = df.to_csv(index=False)
    uri = urllib.parse.quote(csv_text)
    return f"data:text/csv;charset=utf-8,{uri}"

def update_download_link():
    if not resp_df.empty:
        data_uri = make_csv_data_uri(resp_df)
        download_link.value = f'''
        <div style="margin-top:1em">
          <a href="{data_uri}" download="app_responses.csv"
             style="padding:8px 12px;
                    background:#28a745;
                    color:white;
                    text-decoration:none;
                    border-radius:4px;">
            📥 Download all responses
          </a>
        </div>
        '''
    else:
        download_link.value = ''

update_download_link()

current_id = None

# — NAVIGATION & LOGIC (unchanged except for max cap) —
def load_next(_=None):
    global current_id
    if len(resp_df) >= MAX_RATINGS:
        out.value = f"<h3>✅ You’ve completed {MAX_RATINGS} ratings!</h3>"
        btn_submit.disabled = True
        return
    unseen = get_unseen()
    if unseen.empty:
        out.value = "<h3>✅ You’ve rated every text!</h3>"
        btn_submit.disabled = True
        return
    row = unseen.sample(1).iloc[0]
    current_id = row['text_id']
    out.value = f"<b>Text #{current_id}</b><p>{row['processed_tex']}</p>"
    sentiment.value = None

def on_submit(_):
    global resp_df
    if sentiment.value is None:
        return
    resp_df.loc[len(resp_df)] = {
        'text_id': current_id,
        'sentiment': sentiment.value
    }
    resp_df.to_csv(RESPONSES_CSV, index=False)
    update_download_link()
    load_next()

btn_submit.on_click(on_submit)

# — LAYOUT & START —  
display(task, out, sentiment, btn_submit, download_link)
load_next()